In [206]:
import numpy as np
import os
import pulp
import pandas as pd

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [207]:

hours = list(map(str, range(24))) 
prices_columns = list(map(str, range(24))) + ['label']
prices = pd.read_csv('./TestingResults.txt', names = prices_columns)
prices = prices.loc[prices['label']==1]
prices.pop('label')

print(prices)

           0         1         2         3         4         5         6  \
0   4.512853  3.436581  3.682556  3.062718  3.456278  4.028039  3.530470   
4   4.607662  3.688971  3.804653  2.882434  3.247362  3.393615  3.720861   
5   4.550190  3.605517  3.040928  3.653423  3.344234  3.163024  3.834255   
8   4.421347  3.683758  3.923207  3.070730  3.405958  3.375062  4.125457   
12  4.681636  3.359414  3.324270  3.474888  3.662299  3.235766  3.571356   
14  4.309499  3.363291  3.572018  3.111291  3.342271  3.181621  3.781514   
15  4.217611  3.524808  3.259288  3.213414  3.005478  4.093164  4.141625   
16  3.765169  3.428439  3.393895  2.887627  3.482510  4.028138  3.533672   
17  3.748322  3.522640  3.824900  3.729281  3.512599  3.206317  3.881053   
19  4.530964  3.564828  3.125315  2.974750  3.364015  3.457942  4.381639   
23  4.540529  4.073670  3.366615  2.811501  3.481607  3.704785  3.809674   
24  3.952197  3.859600  3.770391  3.084074  3.361085  3.796463  3.786926   
27  4.624424

In [208]:
usertask = pd.read_csv('./UserTask.csv')
usertask.pop('Maximum scheduled energy per hour')

print(usertask)

   User & Task ID  Ready Time  Deadline  Energy Demand
0     user1_task1          20        23              1
1     user1_task2          18        23              2
2     user1_task3          19        21              1
3     user1_task4          12        20              3
4     user1_task5           6        12              3
5     user1_task6          18        20              2
6     user1_task7           4        10              2
7     user1_task8          12        18              2
8     user1_task9           7        14              3
9    user1_task10           8        14              3
10    user2_task1          11        22              2
11    user2_task2           5        11              2
12    user2_task3           5        23              1
13    user2_task4           6        20              3
14    user2_task5          19        19              1
15    user2_task6          18        21              2
16    user2_task7           3        23              3
17    user

In [209]:
usertask.dtypes

User & Task ID    object
Ready Time         int64
Deadline           int64
Energy Demand      int64
dtype: object

In [210]:
# Construct our lp problems
# lp_user1 = pulp.LpProblem("My_LP_Problem", pulp.LpMinimize)
# ...
# lp_user5 = pulp.LpProblem("My_LP_Problem", pulp.LpMinimize)

for user_id in range(1,6):
    print("lp_user%s"%user_id)
    exec("lp_user%s=pulp.LpProblem('My_LP_Problem', pulp.LpMinimize)"%user_id)


lp_user1
lp_user2
lp_user3
lp_user4
lp_user5


In [211]:
# Construct our decision variables
# u1_t1_20 = pulp.LpVariable('u1_t1_20', lowBound=0, upBound=1, cat='Continuous')
# ...
# u5_t10_23 = pulp.LpVariable('u5_t10_23', lowBound=0, upBound=1, cat='Continuous')

task_by_hour = [[[]for j in range(24)] for i in range(5)]
task_by_id = [[[]for j in range(10)] for i in range(5)]

user_id = 1
task_id = 1

for task in usertask.itertuples():
    
    for hour in range(task[2],task[3]+1):
        exec("u%s_t%s_%s=pulp.LpVariable('u%s_t%s_%s', lowBound=0, upBound=1, cat='Continuous')"%(user_id,task_id,hour,user_id,task_id,hour))
        exec("task_by_hour[%s][%s].append(u%s_t%s_%s)"%(user_id-1,hour,user_id,task_id,hour))
        exec("task_by_id[%s][%s].append(u%s_t%s_%s)"%(user_id-1,task_id-1,user_id,task_id,hour))
        
    task_id += 1
    if task_id == 11:
        task_id = 1
        user_id += 1
        task


In [212]:
# define constrain


user_constrain = "0"
task_id = 1
user_id = 1

for task_energy in usertask['Energy Demand']:
    for task in task_by_id[user_id-1][task_id-1]:
        exec("user_constrain += '+ %s'"%task)
        
    exec("user_constrain += ' == %s'"%task_energy)
    exec("lp_user%s += %s"%(user_id,user_constrain))
    task_id += 1
    if task_id == 11:
        task_id = 1
        user_id += 1
    user_constrain = "0"
    



In [216]:
# Construct objective function

tasklist = "0"
user_function = "0"

for price in prices.itertuples():
    for user_id in range(1,6):
        for hour in range(0,24):
            for task in task_by_hour[user_id-1][hour]:
                exec("tasklist += '+ %s'"%task)
            
            exec("user_function += '+ (%s) * %s'"%(tasklist,price[hour+1]))
            tasklist = "0"
       
        exec("lp_user%s += %s"%(user_id,user_function))
        user_function = "0"
        
        exec("lp_user%s.solve()"%user_id)
        # exec("print(pulp.LpStatus[lp_user%s.status])"%user_id)
        
        exec("for variable in lp_user%s.variables(): print('{} = {}'.format(variable.name, variable.varValue))"%user_id)
        
        exec("print(pulp.value(lp_user%s.objective))"%user_id)
        

    


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/reol/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/zy/j2jkps316bzfk9v5fvqrr8rm0000gn/T/2e7d3a45970a40e1a8d1506ee8d08b8f-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/zy/j2jkps316bzfk9v5fvqrr8rm0000gn/T/2e7d3a45970a40e1a8d1506ee8d08b8f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 138 RHS
At line 149 BOUNDS
At line 211 ENDATA
Problem MODEL has 10 rows, 61 columns and 61 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (-2) rows, 54 (-7) columns and 54 (-7) elements
0  Obj 10.247654 Primal inf 19.999992 (8)
8  Obj 100.87087
Optimal - objective value 100.87087
After Postsolve, objective 100.87087, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 100.8708686 - 8 iterations time 0.002, Pr